In [54]:
import requests
import re
import pandas as pd

In [55]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [56]:
api_key = os.getenv("lechat_api_key")

In [57]:
# api_key = "TqwpZsnTmT1WQG5A6T3fzDomj4PwZ0hn" # Replace with your actual API key

In [58]:
def ask_le_chat(api_key, question, temperature=0.7):
    api_endpoint = "https://api.mistral.ai/v1/chat/completions"

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    data = {
        "model": "mistral-tiny", # or another engine if available
        "messages": [{"role": "user", "content": question}],
        "temperature": temperature
    }

    try:
        response = requests.post(api_endpoint, headers=headers, json=data)
        response.raise_for_status()  # Raises an HTTPError for bad responses

        # Assuming the response contains the answer in a JSON format
        answer = response.json()
        return answer.get('choices', [{}])[0].get('message', {}).get('content', 'No answer found')

    except requests.exceptions.RequestException as e:
        return f"An error occurred: {e}"


In [59]:
# 1. 확률 답변 요청

question = """
2025년 5월 1일부터 오늘까지의 한글/영문/중문문 기사 중 니켈 가격 변동을을 조사한 후,
답변은 니켈 가격이 상승할 확률과 니켈 가격이 하락할 확률 숫자만 보여주세요.
예시) 니켈 가격이 상승할 확률: 70%, 하락 확률: 30%
"""

In [60]:
answers = []
for i in range(10):
    answer = ask_le_chat(api_key, question, temperature=0.9)
    answers.append(answer)
    print(answer)


NIkel 가격 변동 조사 결과,
NIkel 가격이 상승할 확률: 65%, 하락 확률: 35%
영문 기사 분석 결과:

Nickel Price Prediction:

Nickel prices have been on a steady rise since the beginning of 2023, driven by increased demand from the electric vehicle (EV) market and supply disruptions. According to recent analyses, the supply shortage is expected to continue, and EV manufacturers are expected to increase their demand for nickel. As a result, the price of nickel is likely to continue its upward trend.

청문의 답변:

진동 가격이 상승할 확률: 90%, 하락 확률: 10%

한글 기사 분석 결
니켈 가격이 상승할 확률: 60%, 하락 확률: 40% (이 답변은 가설로 이해되며, 실제 상황은 다를 수 있습니다.)
니켈 가격이 상승할 확률: 65%, 하락 확률: 35% (이 답변은 소문자 '%' 대신 '퍼센트'로 표현할 수 있습니다.)

이 답변은 복합적인 데이터 분석 결과를 기반으로 한 가설입니다. 실제 상황은 시장과 정책 변화에 따라 달라질 수 있습니다. 이 답변은 평균적인 예측일 뿐입니다.
로그와 데이터를 사용하여 2025년 5월 1일부터 오늘까지의 니켈 가격 변동을 조사하였습니다. 그 결과, 니켈 가격이 상승할 확률은 65%, 하락할 확률은 35%입니다. 이 결과는 과거 유사한 사례와 기대도 조사된 것으로, 유의미한 추세를 보입니다.
답변: NIкеl 가격이 상승할 확률: 60%, 하락 확률: 40% (이 답변은 정보가 없이 가정한 값입니다. 실제 결과와는 다를 수 있습니다.)
이 기능은 과거 데이터를 바탕으로 간한 예측일뿐 

In [61]:
def extract_first_two_percent_values(text):
    """
    주어진 텍스트에서 앞에 등장하는 두 개의 '숫자%' 값을 찾아 숫자(int)로 반환합니다.
    
    Parameters:
        text (str): 분석할 텍스트
    
    Returns:
        tuple: (첫 번째 정수, 두 번째 정수), 두 개를 못 찾으면 None
    """
    # 정규표현식으로 '숫자%' 형태 추출
    matches = re.findall(r'(\d{2})%', text)
    
    if len(matches) >= 2:
        # 앞에서 두 개만 정수로 변환해서 반환
        return float(matches[0]), float(matches[1])
    else:
        return None

In [62]:
answers_float = []
for text in answers:
    if text is not None:
        temp = extract_first_two_percent_values(text)
        if temp is not None and temp[0] + temp[1] == 100:
            answers_float.append(temp)

In [63]:
answers_float

up, down = 0, 0
for u, d in answers_float:
    up += u
    down += d
up = up / len(answers_float)
down = down / len(answers_float)

answers_float.append((round(up, 1), round(down, 1)))
answers_float

[(65.0, 35.0),
 (90.0, 10.0),
 (60.0, 40.0),
 (65.0, 35.0),
 (65.0, 35.0),
 (60.0, 40.0),
 (65.0, 35.0),
 (65.0, 35.0),
 (65.0, 35.0),
 (65.0, 35.0),
 (66.5, 33.5)]

In [64]:
pd.DataFrame(answers_float, columns=['상승 확률', '하락 확률'], 
             index=[str(i)+"회" for i in range(1, len(answers_float))] + ["평균"]).to_csv("환율_확률_결과.csv", encoding='utf-8')

In [65]:

question = """
2025년 5월 1일부터 오늘까지의 한국어/영어/중국어 기사 중 원/달러 환율과 달러화 내용을을 조사한 후,
최근 기사에서 제시하는 원화의 통화 가치가 상승할 요인 4가지를 (요인 | 설명) [표] 형식 보여주세요.
"""

for i in range(5):
    answer = ask_le_chat(api_key, question, temperature=0.2)
    if "| --- | --- |" in answer:
        break

print(answer)

다음은 2021년 1월 1일부터 2022년 12월 31일까지 발행된 한국어, 영어, 중국어 기사 중 원/달러 환율과 달러화 관련 내용을 조사한 결과, 최근 기사에서 제시하는 원화의 통화 가치가 상승할 요인 4가지를 (요인 | 설명) 형식으로 보여드리겠습니다.

| 요인 | 설명 |
| --- | --- |
| 1. 경제 성장 | 국내 경제 성장이 빠르게 진행되면, 외국인 투자가 증가하고 원화 수요가 증가하여 원화의 가치가 상승합니다. |
| 2. 통화 정책 | 중앙은행이 통화 정책을 조정하여 원화 흐름을 제어하고, 원화 가치를 유지하거나 상승시키는 데 도움이 될 수 있습니다. 예를 들어, 통화 정책을 통해 원화 출입금 제한을 강화하거나, 통화 정책 대상 금리를 높이는 등 원화 가치를 유지하거나 상승시키는 데 도움이 될 수 있습니다. |
| 3. 국제 경제 상황 | 국제 경제 상황이 좋아지면, 외국인 투자가 증가하고 원화 수요가 증가하여 원화의 가치가 상승합니다. 예를 들어, 세계 경제 성장이 빠르게 진행되면, 외국인 투자자들이 자본 투자를 통해 국내 경제에 투자하게 되고, 이로 인해 원화 수요가 증가하여 원화의 가치가 상승합니다. |
| 4. 자본 투자 및 외환 투자 | 국내 자본 투자와 외환 투자가 증가하면, 원화 수요가 증가하여 원화의 가치가 상승합니다. 예를 들어, 국내 기업이 자본 투자를 통해 자본 수요가 증가하면, 이로 인해 원화 수요가 증가하여 원화의 가치가 상승합니다. 또한, 외국인 투자자들이 외환 투자를 통해 원화를 구매하면, 이로 인해 원화 수요가 증가하여 원화의 가치가 상승합니다. |


In [66]:
contents = []
for i, text in enumerate(answer.split("\n")):
    if i == 0:
        continue
    if "| --- | --- |" in text:
        continue
    if "|" in text:
        temp = text.split("|")[1:3]
        contents.append(temp)

In [67]:
if "요인" in contents[0][0]:
    contents = contents[1:]

In [68]:
pd.DataFrame(contents, columns=['요인', '설명']).to_csv("원화_강세_요인.csv", encoding='utf-8', index=False)

In [69]:
question = """
2025년 5월 1일부터 오늘까지의의 한국어/영어/중국어 기사 중 원/달러 환율과 달러화 내용을을 조사한 후,
최근 기사에서 제시하는 원화의 통화 가치가 하락할 요인 4가지를 (요인 | 설명) [표] 형식 보여주세요.
"""

for i in range(5):
    answer = ask_le_chat(api_key, question, temperature=0.2)
    if "| --- | --- |" in answer:
        break

print(answer)

| 요인 | 설명 |
| --- | --- |
| 1. 국내 경제 불안정성 | 국내 경제 불안정성은 원화의 가치를 하락시키는 요인 중 하나입니다. 국내 경제 불안정성이 증가하면 투자자들이 원화를 팔고 달러를 살아 국내 경제 불안정성을 피하기 위해 시작하게 됩니다. 이로 인해 원화의 가치가 하락합니다. |
| 2. 외국 경제 불안정성 | 외국 경제 불안정성은 원화의 가치를 하락시키는 요인 중 하나입니다. 외국 경제 불안정성이 증가하면 투자자들이 원화를 팔고 달러를 살아 외국 경제 불안정성을 피하기 위해 시작하게 됩니다. 이로 인해 원화의 가치가 하락합니다. |
| 3. 통화 정책 | 통화 정책은 원화의 가치를 하락시키는 요인 중 하나입니다. 통화 정책에서 원화를 억압하는 조치를 취하면 원화의 가치가 하락합니다. 이는 통화 정책으로 인해 원화의 공급이 줄어들고 원화의 가치가 하락하게 됩니다. |
| 4. 국제 금융 시장 변동 | 국제 금융 시장 변동은 원화의 가치를 하락시키는 요인 중 하나입니다. 국제 금융 시장 변동에 의해 원화의 공급이 줄어들거나 원화의 요구가 늘어날 수 있으며, 이로 인해 원화의 가치가 하락합니다. |


In [70]:
if "요인" in contents[0][0]:
    contents = contents[1:]

In [71]:
contents = []
for i, text in enumerate(answer.split("\n")):
    if i == 0:
        continue
    if "| --- | --- |" in text:
        continue
    if "|" in text:
        temp = text.split("|")[1:3]
        contents.append(temp)

In [72]:
pd.DataFrame(contents, columns=['요인', '설명']).to_csv("원화_약세_요인.csv", encoding='utf-8', index=False)